In [1]:
import csv

# open and read the csv file
f = open("quiz4_routing.csv")
csvfile = csv.DictReader(f, delimiter=',')
headers = csvfile.fieldnames

table = []
for row in csvfile:
    table.append(row)
    
f.close()

# define the set of routes
Routes = headers[:]
Routes.remove('Distribution Center')
Routes.remove('Demand (lbs)')

# read the parameters
Demand = {}        # Demand[j] is the demand for distribution center j
IsInRoute = {}     # IsInRoute[i,j] = 1 if route i includes distribution center j
Length = {}        # Length[i] is the length of route i
for row in table:
    dc = row['Distribution Center']
    if dc != 'n/a':
        Demand[dc] = float(row['Demand (lbs)'])
        for r in Routes:
            if row[r] != '':
                IsInRoute[(r,dc)] = int(row[r])
    else:
        for r in Routes:
            Length[r] = float(row[r])
DCs = Demand.keys()
# Truck capacity
truck_capacity = 30000

In [2]:
from docplex.mp.model import Model
mdl = Model()

In [3]:
# Define decision variables
selected = mdl.binary_var_dict(Routes, name='route')

# Define the objective function
mdl.minimize(mdl.sum(Length[r] * selected[r] for r in Routes))

# Define the constraints
for dc in DCs:
    mdl.add_constraint(mdl.sum(IsInRoute.get((r, dc), 0) * selected[r] for r in Routes) >= 1, f'Demand_{dc}')



In [4]:
# Constraints: Truck capacity must not be exceeded in any route
for r in Routes:
    mdl.add_constraint(mdl.sum(Demand[dc] for dc in DCs if (r, dc) in IsInRoute) <= truck_capacity)


In [5]:
# solve
mdl.solve()
mdl.get_solve_details()

docplex.mp.SolveDetails(time=0.094,status='integer optimal solution')

In [6]:
mdl.print_solution()

objective: 7922.000
status: OPTIMAL_SOLUTION(2)
  route_5=1
  route_6=1
  route_9=1
  route_17=1
  route_24=1
  route_26=1
  route_27=1


In [8]:
# Calculate the total weight loaded on each truck
route_weights = {}
for r in Routes:
    if selected[r].solution_value > 0.5:  # Route is selected
        total_weight = sum(Demand[dc] for dc in DCs if (r, dc) in IsInRoute)
        route_weights[r] = total_weight

# Count the number of routes carrying exactly 30,000 lbs
routes_with_max_weight = [r for r, weight in route_weights.items() if weight == 30000]

print("Total weight on each route:")
for r, weight in route_weights.items():
    print(f"Route {r}: {weight} lbs")

print(f"Number of routes carrying exactly 30,000 lbs: {len(routes_with_max_weight)}")

Total weight on each route:
Route 5: 14000.0 lbs
Route 6: 21000.0 lbs
Route 9: 19000.0 lbs
Route 17: 30000.0 lbs
Route 24: 30000.0 lbs
Route 26: 23000.0 lbs
Route 27: 24000.0 lbs
Number of routes carrying exactly 30,000 lbs: 2


In [ ]:
# Question 9

In [10]:
import csv
from docplex.mp.model import Model

# Load the data from the CSV file
file_path = 'quiz4_routing.csv'
with open(file_path, 'r') as f:
    csvfile = csv.DictReader(f, delimiter=',')
    headers = csvfile.fieldnames
    table = [row for row in csvfile]

# Define the set of routes
Routes = headers[:]
Routes.remove('Distribution Center')
Routes.remove('Demand (lbs)')

# Read the parameters
Demand = {}        # Demand[j] is the demand for distribution center j
IsInRoute = {}     # IsInRoute[i,j] = 1 if route i includes distribution center j
Length = {}        # Length[i] is the length of route i
for row in table:
    dc = row['Distribution Center']
    if dc != 'n/a':
        Demand[dc] = float(row['Demand (lbs)'])
        for r in Routes:
            if row[r] != '':
                IsInRoute[(r, dc)] = int(row[r])
    else:
        for r in Routes:
            Length[r] = float(row[r])

DCs = Demand.keys()
Capacity = 30000  # Truck capacity
MaxExpectedDemand = 27000  # New requirement

# Exclude routes that exceed the new expected demand constraint
ValidRoutes = [r for r in Routes if sum(Demand[dc] for dc in DCs if (r, dc) in IsInRoute) <= MaxExpectedDemand]

# Define the model
mdl = Model()

# Define decision variables
selected = mdl.binary_var_dict(ValidRoutes, name='route')

# Objective function
mdl.minimize(mdl.sum(Length[r] * selected[r] for r in ValidRoutes))

# Constraints: Each distribution center must be visited at least once
for dc in DCs:
    mdl.add_constraint(mdl.sum(IsInRoute.get((r, dc), 0) * selected[r] for r in ValidRoutes) >= 1, f'Demand_{dc}')

# Constraints: Truck capacity must not be exceeded in any route
for r in ValidRoutes:
    mdl.add_constraint(mdl.sum(Demand[dc] for dc in DCs if (r, dc) in IsInRoute) <= Capacity)

# Solve the model
solution = mdl.solve()
solution_details = mdl.get_solve_details()

# Print solution
mdl.print_solution()

# Calculate the total length of the routes used in the optimal solution
total_length = sum(Length[r] * selected[r].solution_value for r in ValidRoutes)
print(f"Total length of the routes used in the optimal solution: {total_length} miles")

objective: 8267.000
status: OPTIMAL_SOLUTION(2)
  route_5=1
  route_6=1
  route_26=1
  route_29=1
  route_30=1
  route_37=1
  route_39=1
Total length of the routes used in the optimal solution: 8267.0 miles
